# Data Cleaning Based on URL 

This notebook was mostly for testing and looking at why articles of other languages ended up in. Most of them seem to have URLs that have endings that indicate this, so at the bottom there is a python function which can verify if a URL should be inserted into the datawarehouse. A simple True False check should account for most of the foreign language articles

## TODO 

Still need to explore ways to clean the body articles 

In [43]:
import pandas as pd
import psycopg2

In [19]:
conn = psycopg2.connect("dbname='cap' user='postgres' host='ec2-35-163-99-253.us-west-2.compute.amazonaws.com' port=9000 password ='secret'")
cursor = conn.cursor()

cursor.execute("SELECT * FROM articles limit 10000") ##This is limited for testing (don't want to query all articles every time I make a change)

data = cursor.fetchall()
data = pd.DataFrame(data)
data.columns = ['site', 'title','author','secondary_authors','published_on','accessed_on','url','body' ,'html','newspaper_keywords','newspaper_summary','id']                 

In [23]:
data

,site,title,author,secondary_authors,published_on,accessed_on,url,body,html,newspaper_keywords,newspaper_summary,id
0,Tech Cruch,Wayfair takes on Pinterest with its own visual...,Sarah Perez,,2017-05-16,2017-05-16 12:31:10.595570,https://techcrunch.com/2017/05/16/wayfair-take...,Pinterest and Google’s recent moves to leverag...,"<div><img src=""https://tctechcrunch2011.files....","{visual,products,wayfair,wayfairs,feature,shop...",Pinterest and Google’s recent moves to leverag...,65756
1,Tech Cruch,CreditHero wants to help fix credit scores for...,Matthew Lynley,,2017-05-15,2017-05-16 12:31:23.991562,https://techcrunch.com/2017/05/15/credithero-w...,With apps like Robinhood and services like Wea...,"<div><img src=""https://tctechcrunch2011.files....","{user,wants,afraid,credit,improve,point,fix,ex...",With apps like Robinhood and services like Wea...,65757
2,Tech Cruch,DeepMind NHS health data deal had ‘no lawful b...,Natasha Lomas,,2017-05-15,2017-05-16 12:31:30.334092,https://techcrunch.com/2017/05/15/deepmind-nhs...,DeepMind’s inaugural data-sharing deal with th...,"<div><img src=""https://tctechcrunch2011.files....","{letter,ndg,royal,patient,care,deal,deepmind,s...",DeepMind’s inaugural data-sharing deal with th...,65758
3,The Next Web,Have questions for your favorite TNW Conferenc...,Caroline Nisenson,,2017-05-16,2017-05-16 12:32:08.105026,https://thenextweb.com/insider/2017/05/16/ques...,Got a question for one or more of the upcoming...,<div><p>Got a question for one or more of the ...,"{conference,questions,covered,answers,youre,fa...",Look no further than our dedicated TNW Answers...,65759
4,The Next Web,How Microsoft wants us all to get creative wit...,Alice Bonasio,"['Technology Writer For', 'Seth Godin']",2017-05-16,2017-05-16 12:32:12.437484,https://thenextweb.com/contributors/2017/05/16...,It is hardly surprising that Artificial Intell...,<div><p>It is hardly surprising that Artificia...,"{microsofts,ai,wants,artificial,vision,service...",It is hardly surprising that Artificial Intell...,65760
5,The Next Web,Making It in the US – Selling B2B from Lithuania,Justas Markus,['Founder Ceo'],2017-05-16,2017-05-16 12:32:18.084658,https://thenextweb.com/contributors/2017/05/16...,Tired of reading about startup success stories...,<div><p>Tired of reading about startup success...,"{business,b2b,lithuanian,efforts,market,team,t...","Companies are easy to talk to, and that helps ...",65761
6,The Next Web,Someone posted the codes to every United cockp...,Rachel Kaser,,2017-05-15,2017-05-16 12:32:23.376356,https://thenextweb.com/insider/2017/05/15/some...,"Over the weekend, someone inadvertently leaked...","<div><p>Over the weekend, someone inadvertentl...","{pilots,united,online,breach,door,posted,inadv...","Over the weekend, someone inadvertently leaked...",65762
7,The Next Web,Disney film held for Bitcoin ransom by hackers,Rachel Kaser,,2017-05-16,2017-05-16 12:32:26.749146,https://thenextweb.com/insider/2017/05/16/disn...,It looks like Disney’s latest film about pirat...,<div><p>It looks like Disney&#8217;s latest fi...,"{hollywood,pirates,film,release,unless,hackers...",It looks like Disney’s latest film about pirat...,65763
8,The Next Web,Local digital marketing is more important than...,John Doherty,['John Doherty Is The Founder Of'],2017-05-16,2017-05-16 12:32:31.333029,https://thenextweb.com/marketing-seo/2017/05/1...,I remember when I started in digital marketing...,"<div><p dir=""ltr"">I remember when I started in...","{digital,business,marketing,facebook,amazon,wo...",The year of local digital marketing?\nSmall lo...,65764
9,The Next Web,4 reasons you should create long-form blog con...,Sabih Javed,,2017-05-16,2017-05-16 12:32:36.959040,https://thenextweb.com/contributors/2017/05/16...,Are you struggling to rank individual blog pos...,<div><p>Are you struggling to rank individual ...,"{user,content,articles,words,article,blog,stud...","However, the average word count for long-form ...",65765


In [29]:
from collections import defaultdict

url_endings = defaultdict(list)

for i, url in enumerate(data['url']):
    url = url.split('//')[-1].split('/')[0].split('.')[-1]
    url_endings[url].append(i)
    
#print(url_endings)

def get_articles_by_end(ending):
    indices = url_endings[ending]
    articles = data.iloc[indices]
    return articles

articles = get_articles_by_end('kr')
articles['body']

1293    ADVERTISEMENT  5월 17일, 피우진 예비역 중령이 국가보훈처장에 임명됐...
1295    WASHINGTON, USA - MAY 10: US President Donald ...
1298    ADVERTISEMENT  5월 17일, 피우진 예비역 중령이 국가보훈처장에 임명됐...
1302    게시됨: 2017년 05월 16일 20시 50분 KST  게시됨: 2017년 05월...
1304    게시됨: 2017년 05월 17일 18시 29분 KST  게시됨: 2017년 05월...
1311    이집트 맥도날드에서 판매하는 이 햄버거에는 '팔라펠'이 패티 대신 들어 있다. '팔...
1315    ADVERTISEMENT    문명고 입학식이 열린 지난 3월2일 오전 경북 경산시...
1319    HOLLYWOOD, : Former US gold medal decathlete B...
1320    게시됨: 2017년 05월 17일 17시 00분 KST  게시됨: 2017년 05월...
1329    게시됨: 2017년 05월 17일 15시 47분 KST  게시됨: 2017년 05월...
1335    ADVERTISEMENT  피우진 예비역 중령이 문재인 정부의 첫 국가보훈처장으로 ...
1353    ADVERTISEMENT  5월 17일, 피우진 예비역 중령이 국가보훈처장에 임명됐...
1380    게시됨: 2017년 05월 17일 15시 18분 KST  게시됨: 2017년 05월...
1387    WASHINGTON, USA - MAY 10: US President Donald ...
1390    ADVERTISEMENT  김동철 국민의당 신임 원내대표가 16일 민주당과 통합론에...
1391    게시됨: 2017년 05월 17일 09시 54분 KST  게시됨: 2017년 05월...
1392    ADVERTISEMENT  피우진 예비역 중령이 문재인 정부의 첫 국가보훈처장으로 ...
1400    게시됨: 2

In [44]:
for key in url_endings:
    print(key)

com

uk
org
au
de
ca
kr
in
jp
fr
es
mx
za
io
google
it
net
edu
gov
gr
co
my
mt


In [5]:
# Want to make sure we get all bad endings
# bad endings tested: kr, cn, ua, dk, ca some french, it, mx, es, fr, de, gr, pk
#articles = get_articles_by_end('gr')
#articles['body']

In [45]:
def is_valid_url( url ):
    invalid_urls = ['kr', 'cn', 'ua', 'dk', 'it', 'mx', 'es', 'fr', 'de', 'gr', 'pk']
    url_ending = url.split('//')[-1].split('/')[0].split('.')[-1]
    if url_ending not in invalid_urls:
        return True
    else:
        return False

In [10]:
print(data['url'][900])
is_valid_url( data['url'][900])


https://www.washingtonpost.com/news/the-fix/wp/2017/04/02/a-judge-rules-trump-may-have-incited-violence-and-trump-again-has-his-own-mouth-to-blame/?tid=pm_pop


True

In [46]:
not_valid = []
for index, row in data.iterrows():
    if is_valid_url(row['url']) is False:
        not_valid.append(row['id'])
        

In [50]:
## Delete all articles where ID is in not_valid
len(not_valid)

557